# Importing Libraries

In [2]:
import import_ipynb
import sys
import os
import operator
import subprocess
import time
import datetime
import pandas as pd 
import numpy as np
import scipy
import pickle

# Data preprocessing

In [5]:
#!/usr/bin/python


####Initializing Usefull Variables

c=1 #  Optional Variable
Dict={} # { Well: { WellDepth : { LogName : LogValue} } }



list_of_las_files=['SUTHRI-1.LAS','BRDW-4-1.LAS'] # List of all .LAS files in directory
#print(list_of_las_files)


####Reading Each LAS file in the directory and saving the values to Dict Variable


for File in list_of_las_files:
    #print("File Number : ", c)
    print(File)

####Reading Each LAS file in the directory and saving the values to Dict Variable


    #print("File Number : ", c)
    FileName=File.split(".LAS")[0]
    header=0
    start=0
    stop=0
    step=0
    Dict[FileName]={}
    readFile=open(File,'r')
    while readFile:
        line= readFile.readline()
        if line == "":
            break
        line=line[:-1]
        if "~Version Information" in line :
            header=1
            start=0
            stop=0
            step=0
            fetchLogName=0
            NumberOfLogs=0
            NumberOfLines=0
            NameOfLog=[]
        if "~Well Information Block" in line :
            header=2
        if "~Parameter Information Block" in line :
            header=3
        if "~Curve Information Block" in line :
            header=4
        if "~A" in line :
            header=5

        if header==2 :
            if "STRT" in line :
                line=line.split()
                start=line[1][:-1]
                print(start)
            if "STOP" in line :
                line=line.split()
                stop=line[1][:-1]
                print(stop)
            if "STEP" in line :
                line=line.split()
                step=line[1][:-1]
                print(step)
        if header==4 :
            if fetchLogName == 0 :
                if "DEPTH.M" in line :
                    fetchLogName =1
            else :
                print(line.split()[0])
                NameOfLog.append(line.split()[0])
                NumberOfLogs=NumberOfLogs+1
        if header ==5 :
            if "~A" in line :
                NumberOfLines=int((NumberOfLogs/7)+1)
                print("Numbers Of Lines of Data :",NumberOfLines)
            #print line
            else :
                Depth=line.split()[0]
                if Depth not in Dict[FileName].keys():
                    Dict[FileName][Depth]={}
                data=[]
                for i in range(0,NumberOfLines):
                    dataline = readFile.readline()
                    dataline = dataline.split()
                    for d in dataline:
                        if d!='':
                            data.append(d)
                logNameIndex=0
                for logValue in data:
                    Dict[FileName][Depth][NameOfLog[logNameIndex]]=logValue

    readFile.close()
    c=c+1
#print(Dict)

SUTHRI-1.LAS
30.6634
3401.9038
0.1524
LITH_NAA2.
LITH_NDA1.
Numbers Of Lines of Data : 1
30.6634
3401.9038
0.1524
LITH_NAA1.
Numbers Of Lines of Data : 1
30.6634
3401.9038
0.1524
LITH_NDA1.
Numbers Of Lines of Data : 1
235.0004
3330.2444
0.1524
CALI.INCH
DT
GR
ILD
ILM
LL3
NPHI.V/V
RHOB.G/CM3
SP
Numbers Of Lines of Data : 2
BRDW-4-1.LAS
2460.0408
2741.0664
0.1524
CFTC.
CNTC.
Numbers Of Lines of Data : 1
2460.0408
2741.0664
0.1524
CFTC.
CNTC.
Numbers Of Lines of Data : 1
2402.4341
3246.8823
0.1524
DRHB.
GR_ARC_RAW.
Numbers Of Lines of Data : 1
2402.4341
3246.8823
0.1524
A34H.OHMM
A40H.OHMM
P16H.OHMM
P34H.OHMM
Numbers Of Lines of Data : 1
2402.4341
3246.8823
0.1524
P34H.OHMM
Numbers Of Lines of Data : 1
2460.0408
2741.0664
0.1524
GR
NPHI.V/V
RHOZ.G/CM3
RLA1.OHMM
RLA2.OHMM
RLA3.OHMM
RLA4.OHMM
RLA5.OHMM
RXOZ.
SP
Numbers Of Lines of Data : 2
2402.4341
3246.8823
0.1524
GR
Numbers Of Lines of Data : 1


In [6]:
import pandas as pd
df=pd.DataFrame(Dict)
df

,SUTHRI-1,BRDW-4-1
30.6634,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
30.8158,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
30.9682,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
31.1206,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
31.2730,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
...,...,...
3246.2727,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."
3246.4251,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."
3246.5775,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."
3246.7299,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."


In [7]:
df.to_pickle('originaldf.pkl')

In [8]:
df1=df.copy()

In [9]:
df1['Base_Depth(meters)'] = df1.index
df1

,SUTHRI-1,BRDW-4-1,Base_Depth(meters)
30.6634,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.6634
30.8158,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.8158
30.9682,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.9682
31.1206,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,31.1206
31.2730,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,31.2730
...,...,...,...
3246.2727,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.2727
3246.4251,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.4251
3246.5775,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.5775
3246.7299,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.7299


In [10]:
df1.index=range(49820)

In [11]:
df1

,SUTHRI-1,BRDW-4-1,Base_Depth(meters)
0,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.6634
1,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.8158
2,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.9682
3,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,31.1206
4,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,31.2730
...,...,...,...
49815,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.2727
49816,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.4251
49817,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.5775
49818,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.7299


In [12]:
df1.dtypes

SUTHRI-1              object
BRDW-4-1              object
Base_Depth(meters)    object
dtype: object

In [13]:
df1['Base_Depth(meters)']= pd.to_numeric(df1['Base_Depth(meters)'],errors = 'coerce')

In [14]:
df1.dtypes

SUTHRI-1               object
BRDW-4-1               object
Base_Depth(meters)    float64
dtype: object

In [15]:
df1

,SUTHRI-1,BRDW-4-1,Base_Depth(meters)
0,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.6634
1,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.8158
2,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,30.9682
3,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,31.1206
4,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN,31.2730
...,...,...,...
49815,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.2727
49816,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.4251
49817,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.5775
49818,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P...",3246.7299


In [20]:
df1.to_pickle('modified_originaldf.pkl')

In [21]:
df2=df1.copy()

In [22]:
df2=df2.drop('Base_Depth(meters)', axis='columns')

In [23]:
df2

,SUTHRI-1,BRDW-4-1
0,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
1,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
2,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
3,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
4,"{'LITH_NAA2.': '33.0000', 'LITH_NAA1.': '33.00...",NaN
...,...,...
49815,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."
49816,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."
49817,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."
49818,NaN,"{'DRHB.': '16.0850', 'A34H.OHMM': '5.4073', 'P..."


In [25]:
suthridf=df2['SUTHRI-1'].apply(pd.Series)

In [26]:
suthridf

,0,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.
0,NaN,NaN,33.0000,33.0000,33.0000
1,NaN,NaN,33.0000,33.0000,33.0000
2,NaN,NaN,33.0000,33.0000,33.0000
3,NaN,NaN,33.0000,33.0000,33.0000
4,NaN,NaN,33.0000,33.0000,33.0000
...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,NaN
49816,NaN,NaN,NaN,NaN,NaN
49817,NaN,NaN,NaN,NaN,NaN
49818,NaN,NaN,NaN,NaN,NaN


In [27]:
suthridf=suthridf.drop(0, axis='columns')

In [28]:
suthridf

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.
0,NaN,33.0000,33.0000,33.0000
1,NaN,33.0000,33.0000,33.0000
2,NaN,33.0000,33.0000,33.0000
3,NaN,33.0000,33.0000,33.0000
4,NaN,33.0000,33.0000,33.0000
...,...,...,...,...
49815,NaN,NaN,NaN,NaN
49816,NaN,NaN,NaN,NaN
49817,NaN,NaN,NaN,NaN
49818,NaN,NaN,NaN,NaN


In [29]:
suthridf.to_pickle('suthridf.pkl')

In [30]:
bdf=df2['BRDW-4-1'].apply(pd.Series)

In [31]:
bdf=bdf.drop(0,axis='columns')

In [32]:
bdf

,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
49815,5.4073,NaN,16.0850,22.9392,5.4073
49816,5.4073,NaN,16.0850,22.9392,5.4073
49817,5.4073,NaN,16.0850,22.9392,5.4073
49818,5.4073,NaN,16.0850,22.9392,5.4073


In [33]:
bdf.to_pickle('bdf.pkl')

In [34]:
depth=df1['Base_Depth(meters)']

In [35]:
depth

0          30.6634
1          30.8158
2          30.9682
3          31.1206
4          31.2730
           ...    
49815    3246.2727
49816    3246.4251
49817    3246.5775
49818    3246.7299
49819    3246.8823
Name: Base_Depth(meters), Length: 49820, dtype: float64

In [36]:
suthridf1=pd.concat([suthridf,depth],axis=1)

In [37]:
suthridf1

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,Base_Depth(meters)
0,NaN,33.0000,33.0000,33.0000,30.6634
1,NaN,33.0000,33.0000,33.0000,30.8158
2,NaN,33.0000,33.0000,33.0000,30.9682
3,NaN,33.0000,33.0000,33.0000,31.1206
4,NaN,33.0000,33.0000,33.0000,31.2730
...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,3246.2727
49816,NaN,NaN,NaN,NaN,3246.4251
49817,NaN,NaN,NaN,NaN,3246.5775
49818,NaN,NaN,NaN,NaN,3246.7299


In [41]:
suthridf1.to_pickle('suthridf1.pkl')

In [39]:
bdf1=pd.concat([bdf,depth],axis=1)

In [40]:
bdf1

,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Base_Depth(meters)
0,NaN,NaN,NaN,NaN,NaN,30.6634
1,NaN,NaN,NaN,NaN,NaN,30.8158
2,NaN,NaN,NaN,NaN,NaN,30.9682
3,NaN,NaN,NaN,NaN,NaN,31.1206
4,NaN,NaN,NaN,NaN,NaN,31.2730
...,...,...,...,...,...,...
49815,5.4073,NaN,16.0850,22.9392,5.4073,3246.2727
49816,5.4073,NaN,16.0850,22.9392,5.4073,3246.4251
49817,5.4073,NaN,16.0850,22.9392,5.4073,3246.5775
49818,5.4073,NaN,16.0850,22.9392,5.4073,3246.7299


In [42]:
bdf1.to_pickle('bdf1.pkl')

In [47]:
litho_file=pd.read_csv("C:/Users/Aditi/Desktop/Studies/Internship/ONGC internship/work/ML-Project_Intern(1)/All_well_Litho_Class.csv")

In [48]:
litho_file.shape

(63613, 4)

In [49]:
litho_file

,Common_Well_Name,Top_Depth(meters),Base_Depth(meters),Litho_Class
0,R-62-1,2549.45340,2553.43120,LIMESTONE
1,R-62-1,2681.12350,2687.66430,LIMESTONE
2,R-62-1,2980.91200,2989.37350,SILTSTONE
3,R-62-1,2669.04760,2681.12350,SiltyShale
4,R-62-1,2563.37600,2568.34840,LIMESTONE
...,...,...,...,...
63608,SM-1-1,938.77997,944.03674,SiltyShale
63609,SM-1-1,944.03674,1081.71200,SHALE
63610,SM-1-1,1081.71200,1090.43500,SiltyShale
63611,SM-1-1,1090.43500,1094.95690,SILTSTONE


In [50]:
litho_file.dtypes

Common_Well_Name       object
Top_Depth(meters)     float64
Base_Depth(meters)    float64
Litho_Class            object
dtype: object

In [51]:
litho_file.to_pickle('litho_file.pkl')

In [52]:
litho_file1=pd.DataFrame(litho_file, columns=['Common_Well_Name','Top_Depth(meters)','Base_Depth(meters)','Litho_Class'])

In [53]:
litho_file1

,Common_Well_Name,Top_Depth(meters),Base_Depth(meters),Litho_Class
0,R-62-1,2549.45340,2553.43120,LIMESTONE
1,R-62-1,2681.12350,2687.66430,LIMESTONE
2,R-62-1,2980.91200,2989.37350,SILTSTONE
3,R-62-1,2669.04760,2681.12350,SiltyShale
4,R-62-1,2563.37600,2568.34840,LIMESTONE
...,...,...,...,...
63608,SM-1-1,938.77997,944.03674,SiltyShale
63609,SM-1-1,944.03674,1081.71200,SHALE
63610,SM-1-1,1081.71200,1090.43500,SiltyShale
63611,SM-1-1,1090.43500,1094.95690,SILTSTONE


In [54]:
litho_file_SUTHRI = litho_file1.loc[litho_file1['Common_Well_Name'] == 'SUTHRI-1']

In [55]:
litho_file_SUTHRI

,Common_Well_Name,Top_Depth(meters),Base_Depth(meters),Litho_Class
48620,SUTHRI-1,2519.0083,2527.4653,LIMESTONE
48621,SUTHRI-1,2527.4653,2530.7747,MARL
48622,SUTHRI-1,2530.7747,2570.1697,LIMESTONE
48623,SUTHRI-1,2570.1697,2571.9346,MARL
48624,SUTHRI-1,2571.9346,2576.6665,LIMESTONE
...,...,...,...,...
49449,SUTHRI-1,2932.3157,2941.7036,LIMESTONE
49450,SUTHRI-1,2941.7036,2944.8330,SILTSTONE
49451,SUTHRI-1,2944.8330,2954.7424,LIMESTONE
49452,SUTHRI-1,2954.7424,2955.7856,SILTSTONE


In [56]:
litho_file_SUTHRI.to_pickle('litho_file_SUTHRI.pkl')

In [57]:
litho_file_BRDW = litho_file1.loc[litho_file1['Common_Well_Name'] == 'BRDW-4-1']

In [58]:
litho_file_BRDW

,Common_Well_Name,Top_Depth(meters),Base_Depth(meters),Litho_Class
26749,BRDW-4-1,0.0100,2378.7585,WATER
26750,BRDW-4-1,2378.7585,2404.7388,CLAY
26751,BRDW-4-1,2404.7388,2415.3645,CLAYSTONE
26752,BRDW-4-1,2415.3645,2416.3645,LIMESTONE
26753,BRDW-4-1,2416.3645,2425.9905,CLAYSTONE
...,...,...,...,...
26820,BRDW-4-1,3181.8755,3182.8755,LIMESTONE
26821,BRDW-4-1,3182.8755,3189.6638,CLAYSTONE
26822,BRDW-4-1,3189.6638,3190.6638,LIMESTONE
26823,BRDW-4-1,3190.6638,3208.3218,CLAYSTONE


In [59]:
litho_file_BRDW.to_pickle('litho_file_BRDW.pkl')

In [ ]:
#df13=df13.drop('Litho',axis='columns')

In [63]:
d3=pd.read_pickle('suthridf1.pkl')

In [64]:
d3 = d3.rename(columns={"Base_Depth(meters)":"Depth"})
d3

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,Depth
0,NaN,33.0000,33.0000,33.0000,30.6634
1,NaN,33.0000,33.0000,33.0000,30.8158
2,NaN,33.0000,33.0000,33.0000,30.9682
3,NaN,33.0000,33.0000,33.0000,31.1206
4,NaN,33.0000,33.0000,33.0000,31.2730
...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,3246.2727
49816,NaN,NaN,NaN,NaN,3246.4251
49817,NaN,NaN,NaN,NaN,3246.5775
49818,NaN,NaN,NaN,NaN,3246.7299


In [120]:
d4=pd.read_pickle('litho_file_SUTHRI.pkl')

In [123]:
d4

,Common_Well_Name,Top_Depth(meters),Base_Depth(meters),Litho_Class
48620,SUTHRI-1,2519.0083,2527.4653,LIMESTONE
48621,SUTHRI-1,2527.4653,2530.7747,MARL
48622,SUTHRI-1,2530.7747,2570.1697,LIMESTONE
48623,SUTHRI-1,2570.1697,2571.9346,MARL
48624,SUTHRI-1,2571.9346,2576.6665,LIMESTONE
...,...,...,...,...
49449,SUTHRI-1,2932.3157,2941.7036,LIMESTONE
49450,SUTHRI-1,2941.7036,2944.8330,SILTSTONE
49451,SUTHRI-1,2944.8330,2954.7424,LIMESTONE
49452,SUTHRI-1,2954.7424,2955.7856,SILTSTONE


In [125]:
d3['Litho']=np.nan
pd.options.mode.chained_assignment = None 
for i in d3.index:
    depth=d3.loc[i]['Depth']
    
    for j,k,l in zip(d4['Base_Depth(meters)'], d4['Top_Depth(meters)'],d4['Litho_Class']):
        #print(j,k,l)
        #print(type(i),type(j),type(k))
        if depth >= k and depth <= j:
            #print(depth,j,d2['Litho_Class'])
            d3.loc[i,'Litho']=l
            #print(d1.loc[i]['Litho'])

In [126]:
d3

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,Depth,Litho
0,NaN,33.0000,33.0000,33.0000,30.6634,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,30.8158,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,30.9682,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,31.1206,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,31.2730,CLAYSTONE_KNP
...,...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,3246.2727,INTRUSIVE IGNE_KNP
49816,NaN,NaN,NaN,NaN,3246.4251,INTRUSIVE IGNE_KNP
49817,NaN,NaN,NaN,NaN,3246.5775,INTRUSIVE IGNE_KNP
49818,NaN,NaN,NaN,NaN,3246.7299,INTRUSIVE IGNE_KNP


In [133]:
d3=d3.drop('Depth',axis='columns')

In [134]:
d3

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,Litho
0,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP
49816,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP
49817,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP
49818,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP


In [138]:
d3.to_pickle('finalsuthri.pkl')

In [171]:
d5=pd.read_pickle('bdf1.pkl')

In [172]:
d5=d5.rename(columns={"Base_Depth(meters)":"Depth"})

In [173]:
d5

,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Depth
0,NaN,NaN,NaN,NaN,NaN,30.6634
1,NaN,NaN,NaN,NaN,NaN,30.8158
2,NaN,NaN,NaN,NaN,NaN,30.9682
3,NaN,NaN,NaN,NaN,NaN,31.1206
4,NaN,NaN,NaN,NaN,NaN,31.2730
...,...,...,...,...,...,...
49815,5.4073,NaN,16.0850,22.9392,5.4073,3246.2727
49816,5.4073,NaN,16.0850,22.9392,5.4073,3246.4251
49817,5.4073,NaN,16.0850,22.9392,5.4073,3246.5775
49818,5.4073,NaN,16.0850,22.9392,5.4073,3246.7299


In [174]:
d6=pd.read_pickle('litho_file_BRDW.pkl')

In [175]:
d5['Litho']=np.nan
pd.options.mode.chained_assignment = None 
for i in d5.index:
    depth=d5.loc[i]['Depth']
    
    for j,k,l in zip(d6['Base_Depth(meters)'], d6['Top_Depth(meters)'],d6['Litho_Class']):
        #print(j,k,l)
        #print(type(i),type(j),type(k))
        if depth >= k and depth <= j:
            #print(depth,j,d2['Litho_Class'])
            d5.loc[i,'Litho']=l
            #print(d1.loc[i]['Litho'])

In [176]:
d5

,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Depth,Litho
0,NaN,NaN,NaN,NaN,NaN,30.6634,WATER
1,NaN,NaN,NaN,NaN,NaN,30.8158,WATER
2,NaN,NaN,NaN,NaN,NaN,30.9682,WATER
3,NaN,NaN,NaN,NaN,NaN,31.1206,WATER
4,NaN,NaN,NaN,NaN,NaN,31.2730,WATER
...,...,...,...,...,...,...,...
49815,5.4073,NaN,16.0850,22.9392,5.4073,3246.2727,TRAP_TD67
49816,5.4073,NaN,16.0850,22.9392,5.4073,3246.4251,TRAP_TD67
49817,5.4073,NaN,16.0850,22.9392,5.4073,3246.5775,TRAP_TD67
49818,5.4073,NaN,16.0850,22.9392,5.4073,3246.7299,TRAP_TD67


In [177]:
d5=d5.drop('Depth',axis='columns')

In [178]:
d5

,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,NaN,NaN,NaN,NaN,WATER
1,NaN,NaN,NaN,NaN,NaN,WATER
2,NaN,NaN,NaN,NaN,NaN,WATER
3,NaN,NaN,NaN,NaN,NaN,WATER
4,NaN,NaN,NaN,NaN,NaN,WATER
...,...,...,...,...,...,...
49815,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49816,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49817,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49818,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


In [179]:
d5.to_pickle('finalbrdw.pkl')

In [180]:
d3

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,Litho
0,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP
...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP
49816,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP
49817,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP
49818,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP


In [181]:
d3.iloc[:,4].unique()

array(['CLAYSTONE_KNP', 'SANDSTONE_KNP', 'LIMESTONE_KNP', 'LIMESTONE',
       'SHALE', 'SANDSTONE', 'CLAY', 'TRAP_TD67', 'SHALY FINE SAND',
       'SILTSTONE', 'SILTYSHALE_KNP', 'CLAYSTONE', 'MARL',
       'SHALY LIMESTONE', 'DOLERITE', 'OOLITIC LIMESTONE',
       'INTRUSIVE IGNE_KNP', nan], dtype=object)

In [182]:
d5

,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,NaN,NaN,NaN,NaN,WATER
1,NaN,NaN,NaN,NaN,NaN,WATER
2,NaN,NaN,NaN,NaN,NaN,WATER
3,NaN,NaN,NaN,NaN,NaN,WATER
4,NaN,NaN,NaN,NaN,NaN,WATER
...,...,...,...,...,...,...
49815,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49816,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49817,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49818,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


In [183]:
d5.iloc[:,5].unique()

array(['WATER', 'CLAY', 'CLAYSTONE', 'LIMESTONE', 'TRAP_TD67', nan],
      dtype=object)

In [184]:
d7=pd.concat([d3,d5],axis=1)

In [185]:
d7

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,Litho,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN,WATER
1,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN,WATER
2,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN,WATER
3,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN,WATER
4,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN,WATER
...,...,...,...,...,...,...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49816,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49817,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49818,NaN,NaN,NaN,NaN,INTRUSIVE IGNE_KNP,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


In [187]:
d8=pd.concat([d3,d5])
d8

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,Litho,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM
0,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN
1,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN
2,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN
3,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN
4,NaN,33.0000,33.0000,33.0000,CLAYSTONE_KNP,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,TRAP_TD67,5.4073,NaN,16.0850,22.9392,5.4073
49816,NaN,NaN,NaN,NaN,TRAP_TD67,5.4073,NaN,16.0850,22.9392,5.4073
49817,NaN,NaN,NaN,NaN,TRAP_TD67,5.4073,NaN,16.0850,22.9392,5.4073
49818,NaN,NaN,NaN,NaN,TRAP_TD67,5.4073,NaN,16.0850,22.9392,5.4073


In [188]:
last_col=d8.pop('Litho')

In [189]:
d8.insert(9,'Litho',last_col)

In [190]:
d8

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
...,...,...,...,...,...,...,...,...,...,...
49815,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49816,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49817,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
49818,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


In [195]:
d8.index=range(99640)

In [196]:
d8.index

RangeIndex(start=0, stop=99640, step=1)

In [197]:
d8

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
...,...,...,...,...,...,...,...,...,...,...
99635,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99636,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99637,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99638,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


In [198]:
d8.to_pickle('final1.pkl')

In [217]:
d8=pd.read_pickle('final1.pkl')

# Removing NaN values

In [218]:
d8 = d8.dropna(how='all')

In [219]:
d8

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
...,...,...,...,...,...,...,...,...,...,...
99635,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99636,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99637,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99638,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


In [220]:
d8 = d8.dropna(subset=['Litho'])

In [222]:
d8

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
...,...,...,...,...,...,...,...,...,...,...
99635,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99636,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99637,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99638,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


In [223]:
d8.to_pickle('final2.pkl')

In [225]:
d9=d8.copy()
d9

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
1,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
2,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
3,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
4,NaN,33.0000,33.0000,33.0000,NaN,NaN,NaN,NaN,NaN,CLAYSTONE_KNP
...,...,...,...,...,...,...,...,...,...,...
99635,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99636,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99637,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67
99638,NaN,NaN,NaN,NaN,5.4073,NaN,16.0850,22.9392,5.4073,TRAP_TD67


# Preparing training and testing data

In [35]:
d9=pd.read_pickle('final2.pkl')
d10=d9.copy()

In [36]:
d10.fillna(-999.2500,inplace=True)
d10

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM,Litho
0,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25,CLAYSTONE_KNP
1,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25,CLAYSTONE_KNP
2,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25,CLAYSTONE_KNP
3,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25,CLAYSTONE_KNP
4,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25,CLAYSTONE_KNP
...,...,...,...,...,...,...,...,...,...,...
99635,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073,TRAP_TD67
99636,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073,TRAP_TD67
99637,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073,TRAP_TD67
99638,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073,TRAP_TD67


In [37]:
x=d10.drop('Litho',axis='columns')

In [38]:
x

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM
0,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25
1,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25
2,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25
3,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25
4,-999.25,33.0000,33.0000,33.0000,-999.25,-999.25,-999.25,-999.25,-999.25
...,...,...,...,...,...,...,...,...,...
99635,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073
99636,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073
99637,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073
99638,-999.25,-999.25,-999.25,-999.25,5.4073,-999.25,16.0850,22.9392,5.4073


In [39]:
y=d10['Litho']

In [40]:
y

0        CLAYSTONE_KNP
1        CLAYSTONE_KNP
2        CLAYSTONE_KNP
3        CLAYSTONE_KNP
4        CLAYSTONE_KNP
             ...      
99635        TRAP_TD67
99636        TRAP_TD67
99637        TRAP_TD67
99638        TRAP_TD67
99639        TRAP_TD67
Name: Litho, Length: 98103, dtype: object

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.1,random_state=50)

In [52]:
x_train

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM
50382,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
76028,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
60822,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
32211,51.7000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
52855,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
...,...,...,...,...,...,...,...,...,...
55379,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
59371,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
48430,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
22650,7.9000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25


In [53]:
x_test

,CALI.INCH,LITH_NAA1.,LITH_NAA2.,LITH_NDA1.,A34H.OHMM,CFTC.,DRHB.,GR,P34H.OHMM
59778,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
87654,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
33505,21.7000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
37541,69.8000,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
53475,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
...,...,...,...,...,...,...,...,...,...
6360,-999.25,16.0000,16.0000,16.0000,-999.25,-999.25,-999.25,-999.25,-999.25
4723,-999.25,16.0000,16.0000,16.0000,-999.25,-999.25,-999.25,-999.25,-999.25
55300,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25
50451,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25,-999.25


In [54]:
y_test

59778             WATER
87654         CLAYSTONE
33505    SILTYSHALE_KNP
37541         LIMESTONE
53475             WATER
              ...      
6360          TRAP_TD67
4723          TRAP_TD67
55300             WATER
50451             WATER
51312             WATER
Name: Litho, Length: 9811, dtype: object

# Model Training

In [55]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=50, multi_class='ovr')

In [56]:
model=clf.fit(x_train,y_train)

# Model Performance

In [57]:
new_test=[[21.7000,0,0,0,0,0,0,0,0]]

In [58]:
model.predict(new_test)

array(['SILTYSHALE_KNP'], dtype=object)

In [62]:
model.score(x_test,y_test)

0.46559983691774537